In [1]:
import numpy as np
import pandas as pd


In [2]:
from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [5]:
df.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
571,1,1,female,53.0,2,0,51.4792,S
164,0,3,male,1.0,4,1,39.6875,S
309,1,1,female,30.0,0,0,56.9292,C
591,1,1,female,52.0,1,0,78.2667,C
209,1,1,male,40.0,0,0,31.0000,C


In [6]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [8]:
X_train.shape,X_test.shape

((712, 7), (179, 7))

In [9]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
764,3,male,16.0,0,0,7.775,S
808,2,male,39.0,0,0,13.000,S
412,1,female,33.0,1,0,90.000,Q
344,2,male,36.0,0,0,13.000,S
716,1,female,38.0,0,0,227.525,C


In [10]:
numerical_features = ['Age','Fare']
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])



In [11]:
categorical_features = ['Embarked','Sex']
categorical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ohe',OneHotEncoder(handle_unknown='ignore'))
])

In [12]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num',numerical_transformer,numerical_features),
        ('cat',categorical_transformer,categorical_features)
    ]
)

In [13]:
clf= Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',LogisticRegression())
])

In [14]:
from sklearn import set_config
set_config(display='diagram')

In [15]:
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex'])])),
                ('classifier', LogisticRegression())])

In [21]:
param_grid = {
    'preprocessor__num__imputer__strategy' :['mean','median'],
    'preprocessor__cat__imputer__strategy' :['most_frequent','constant'],
    'classifier__C' :[0.1,1.0,10,100]
}

grid_search = GridSearchCV(clf,param_grid,cv=10)

In [22]:
grid_search.fit(X_train,y_train)

print(f"Best params:")
print(grid_search.best_params_)

Best params:
{'classifier__C': 1.0, 'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__num__imputer__strategy': 'mean'}


In [23]:
print(f"Internal CV sroce: {grid_search.best_score_:.3f}")

Internal CV sroce: 0.787


In [24]:
import pandas as pd

In [25]:
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values("mean_test_score", ascending=False)
cv_results[['param_classifier__C','param_preprocessor__cat__imputer__strategy','param_preprocessor__num__imputer__strategy','mean_test_score']]


,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,mean_test_score
4,1.0,most_frequent,mean,0.786620
6,1.0,constant,mean,0.786620
8,10,most_frequent,mean,0.786620
9,10,most_frequent,median,0.786620
10,10,constant,mean,0.786620
11,10,constant,median,0.786620
12,100,most_frequent,mean,0.786620
13,100,most_frequent,median,0.786620
14,100,constant,mean,0.786620
15,100,constant,median,0.786620
